In [2]:
%matplotlib inline

In [3]:
import os
import random
import pickle
import numpy as np
import tensorflow as tf

from model import c2v
from data import build_vocab

# config
config = {
    'batch_size'      : 16,
    'embed_size'      : 200,
    'neg_sample_size' : 100,
}

# data
data, unique_neg_data, idx2word, word2idx, vocab = build_vocab()

name2idx = dict([(name, idx) for idx, name in enumerate(data.keys())])
idx2name = dict([(idx, name) for idx, name in enumerate(data.keys())])

vocab_size = len(idx2word)
character_size = len(data)

config['vocab_size'] = vocab_size
config['character_size'] = character_size

# model
pos_x, pos_y, neg_y, train, loss, nembed, nearby_character, nearby_val, nearby_idx = c2v(config)

# train
step_size = 100000
learning_rate = 0.025
window_size = 5
min_count = 5
subsample = 1e-3

batch_idx = 0
data_idx = {}

for i in xrange(len(data)):
    data_idx[i] = 0

def generate_batch(config, data, unique_neg_data):
    global batch_idx, data_idx

    batch_size = config['batch_size']
    neg_sample_size = config['neg_sample_size']

    batch = data.values()[batch_idx]
    neg_batch = unique_neg_data.values()[batch_idx]
    idx = data_idx[batch_idx]

    data_pos_x = np.ones(batch_size) * batch_idx
    data_pos_y = np.ndarray(shape=batch_size, dtype=np.int32)
    data_neg_y = np.ndarray(shape=neg_sample_size, dtype=np.int32)

    for i in xrange(batch_size):
        data_pos_y[i] = batch[idx]
        idx = (idx + 1) % len(batch)

    for i, neg_y_idx in enumerate(random.sample(set(neg_batch), neg_sample_size)):
        data_neg_y[i] = neg_y_idx

    data_idx[batch_idx] = idx
    batch_idx = (batch_idx + 1) % len(data)

    return data_pos_x, data_pos_y, data_neg_y

batch_pkl = "./batch.pkl"

if os.path.isfile(batch_pkl):
    batch = pickle.load(open(batch_pkl))
else:
    batch = []
    for i in xrange(step_size):
        batch.append(generate_batch(config, data, unique_neg_data))

    pickle.dump(batch, open(batch_pkl, "wb"))

sess = tf.InteractiveSession()

#with tf.Session() as sess:
if True:
    sess.run(tf.initialize_all_variables())
    average_loss = 0

    batch_size = config['batch_size']
    neg_sample_size = config['neg_sample_size']

    for step in xrange(step_size):
        data_pos_x, data_pos_y, data_neg_y = batch[step]
        feed_dict = {pos_x: data_pos_x, pos_y: np.reshape(data_pos_y, (batch_size)), neg_y: np.reshape(data_neg_y, (neg_sample_size))}
        _, loss_val = sess.run([train, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000

            print("Average loss at step ", step, ": ", average_loss)
            acharacter_sizeverage_loss = 0

def nearby(words, num=20):
    ids = np.array([name2idx.get(x, 0) for x in words])
    vals, idx = sess.run(
        [nearby_val, nearby_idx], {nearby_character: ids})
    for i in xrange(len(words)):
        print(words[i])
        print()
        for (neighbor, distance) in zip(idx[i, :num], vals[i, :num]):
            print("%-20s %6.4f" % (idx2name[neighbor], distance))

def nearby_with_idx(idxs, num=20):
    ids = np.array(idxs)
    vals, idx = sess.run(
        [nearby_val, nearby_idx], {nearby_character: ids})
    for i in xrange(len(idxs)):
        print(idx2name[idxs[i]])
        print()
        for (neighbor, distance) in zip(idx[i, :num], vals[i, :num]):
            print("%-20s %6.4f" % (idx2name[neighbor], distance))

('Average loss at step ', 0, ': ', 70.013839721679688)
('Average loss at step ', 2000, ': ', 69.458465099334717)
('Average loss at step ', 4000, ': ', 68.306635444402701)
('Average loss at step ', 6000, ': ', 67.170784470828892)
('Average loss at step ', 8000, ': ', 66.036558750389716)
('Average loss at step ', 10000, ': ', 64.896763282121782)
('Average loss at step ', 12000, ': ', 63.725812629916824)
('Average loss at step ', 14000, ': ', 62.4931471661111)
('Average loss at step ', 16000, ': ', 61.146574198857714)
('Average loss at step ', 18000, ': ', 59.613447392552914)
('Average loss at step ', 20000, ': ', 57.801283492189924)
('Average loss at step ', 22000, ': ', 55.579718240088987)
('Average loss at step ', 24000, ': ', 52.787522387806568)
('Average loss at step ', 26000, ': ', 49.27575272611395)
('Average loss at step ', 28000, ': ', 45.029972223134294)
('Average loss at step ', 30000, ': ', 40.098238086773797)
('Average loss at step ', 32000, ': ', 34.770710144090685)
('Averag

In [4]:
embed_size = config.get('embed_size', 200)
batch_size = config.get('batch_size', 16)
neg_sample_size = config.get('neg_sample_size', 100)

nearby_emb = tf.reshape(tf.gather(nembed, nearby_character), [1, embed_size])
nearby_dist = tf.matmul(nearby_emb, nembed, transpose_b=True)

In [5]:
nearby_dist.eval({nearby_character: range(character_size)})

InvalidArgumentError: Input has 6400 values, which isn't the same as 200
	 [[Node: Reshape_2 = Reshape[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Gather_1, Reshape_2/shape)]]
Caused by op u'Reshape_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 160, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", line

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)

def plot_with_labels(low_dim_embs, labels, filename='p2v.png'):
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

plot_only = 500
plot_with_labels(tsne.fit_transform(nembed.eval()[:plot_only,:]), data.keys())